In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.impute import SimpleImputer

from rdkit import Chem, DataStructs
from rdkit.Chem import Descriptors,Crippen,DetectChemistryProblems, PandasTools

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import matplotlib.cm as cm
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans

In [2]:
def get_desc(mol):
    desc_dict = {}
    for desc_name, desc_calc in Descriptors._descList:
        try:
            desc_value = desc_calc(mol)
        except:
            desc_value = None
        desc_dict[desc_name] = desc_value
    
    return desc_dict            

In [3]:
df = pd.read_csv('EGFR_Chembl_clean.csv')
df.head()

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties,Action Type,pIC50,Standardized Smiles,Molecule
0,CHEMBL271410,NaN,NaN,378.48,0.0,3.21,"126, page S27 table 1",Cc1cccc(Nc2ncnc3ccncc23)c1NCCCN1CCOCC1,IC50,'=',...,1,Scientific Literature,J Med Chem,2008.0,NaN,NaN,NaN,8.03,Cc1cccc(Nc2ncnc3ccncc23)c1NCCCN1CCOCC1,<rdkit.Chem.rdchem.Mol object at 0x000001B30B8...
1,CHEMBL411243,NaN,NaN,429.37,0.0,4.67,"109, page S26 table 1",CN(C)CCCCCNc1c(Br)cccc1Nc1ncnc2ccncc12,IC50,'=',...,1,Scientific Literature,J Med Chem,2008.0,NaN,NaN,NaN,8.07,CN(C)CCCCCNc1c(Br)cccc1Nc1ncnc2ccncc12,<rdkit.Chem.rdchem.Mol object at 0x000001B30B8...
2,CHEMBL270713,NaN,NaN,387.29,0.0,3.50,"106, page S26 table 1",CN(C)CCNc1c(Br)cccc1Nc1ncnc2ccncc12,IC50,'=',...,1,Scientific Literature,J Med Chem,2008.0,NaN,NaN,NaN,7.34,CN(C)CCNc1c(Br)cccc1Nc1ncnc2ccncc12,<rdkit.Chem.rdchem.Mol object at 0x000001B30B8...
3,CHEMBL54475,NaN,NaN,301.15,0.0,3.53,"78, page S25 table 1",Brc1cccc(Nc2ncnc3ncccc23)c1,IC50,'=',...,1,Scientific Literature,J Med Chem,2008.0,NaN,NaN,NaN,6.16,Brc1cccc(Nc2ncnc3ncccc23)c1,<rdkit.Chem.rdchem.Mol object at 0x000001B30B8...
4,CHEMBL405772,NaN,NaN,251.29,0.0,2.66,"72, page S25 table 1",Cc1cccc(Nc2ncnc3ccncc23)c1N,IC50,'=',...,1,Scientific Literature,J Med Chem,2008.0,NaN,NaN,NaN,7.39,Cc1cccc(Nc2ncnc3ccncc23)c1N,<rdkit.Chem.rdchem.Mol object at 0x000001B30B8...


In [4]:
df = df[['Molecule ChEMBL ID','Standardized Smiles','pIC50']]
PandasTools.AddMoleculeColumnToFrame(df,'Standardized Smiles', 'Molecule', includeFingerprints = False)

In [ ]:
for i in range(df.shape[0]):
    mol = df.loc[i,'Molecule']
    desc_dict = get_desc(mol)
    df.loc[i,list(desc_dict.keys())] = desc_dict.values()
    
df.head()

In [ ]:
list(df.columns[4:])

In [ ]:
df.shape

In [ ]:
df.dropna(axis = 'columns', how = 'all', inplace = True)
desc_array = df.iloc[:,4:].values

if np.isnan(desc_array).any():
    imp = SimpleImputer(missing_values = np.nan, strategy = 'mean')
    imp = imp.fit(desc_array)
    desc_array = imp.transform(desc_array)
desc_array.shape

In [ ]:
sc = StandardScaler()
std_array = sc.fit_transform(desc_array)

pca = PCA(n_components = 0.85) #Get components to explain 85% of the explained variance
array_pca = pca.fit_transform(std_array)
var = pca.explained_variance_ratio_
len(var)

In [ ]:
cum_var = np.cumsum(var)

plt.bar(range(1,49), var, alpha = 0.5, align = 'center', label = 'Explained variance by singular value')
plt.step(range(1,49), cum_var, where = 'mid', label = 'Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Pricipal component')
plt.ylim(0.0,0.90)
plt.tight_layout()
plt.show

In [ ]:
pca_df = pd.DataFrame(data = array_pca, index = df.index)
pca_df.columns = [f'PCA_{x + 1}' for x in pca_df.columns]
pca_df[['Molecule ChEMBL ID','Standardized Smiles','pIC50']] = df[['Molecule ChEMBL ID','Standardized Smiles','pIC50']]

In [ ]:
pca_df.to_csv('egfr_pca.csv', index = False)

Perform k-means clustering